# **Run the following cells to install dependencies and define the necessary functions**

In [1]:
!pip install groq requests beautifulsoup4 gtts IPython googlesearch-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [2]:
import os
import requests
from bs4 import BeautifulSoup
from groq import Groq
from gtts import gTTS
from IPython.display import Audio, display
from googlesearch import search

In [3]:
# Step 1: Set up the Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the Groq client
client = Groq()

In [4]:
# Step 2: Customization options
def get_user_preferences():
    print("Please select your news preferences:")
    country = input("Enter country code (e.g., 'us' for USA, 'in' for India): ").lower()
    category = input("Enter news category (e.g., 'technology', 'business', 'entertainment', 'health'): ").lower()
    source = input("Enter specific news source (e.g., 'bbc-news', 'the-hindu') or leave blank for default: ").lower()
    role = input("Enter your role (e.g., 'student', 'lawyer', 'politician') for personalized news: ").lower()

    return country, category, source, role

In [5]:
# Step 3: Perform Google Search based on user preferences
def google_search_news(query):
    try:
        # Perform Google search with up to 5 results
        search_results = search(query, num_results=5)
        return [{"title": result.split("/")[2], "url": result} for result in search_results]
    except Exception as e:
        print("Error during Google search:", e)
        return []

In [6]:
# Step 4: Build a query for Google search based on user preferences
def build_query(country, category, source, role):
    query = f"{category} news from {source if source else country} {role}"
    return query

In [7]:
# Step 5: Scrape and summarize an article
def scrape_and_summarize_article(article_url):
    response = requests.get(article_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Assuming the article's content is within <p> tags
        paragraphs = soup.find_all('p')
        article_text = "\n".join([para.get_text() for para in paragraphs])

        print("Extracted text from article:\n", article_text)

        # Summarize the article using Groq LLaMA model
        summary = summarize_article_groq(article_text)
        print("\nSummary:\n", summary)
        return summary
    else:
        print("Failed to retrieve the article. Please check the URL.")
        return ""

In [8]:
# Step 6: Summarization using Groq's LLaMA-3.1-70B model
def summarize_article_groq(article_text):
    # Define the task for the LLaMA model via Groq API
    message = {
        "role": "user",
        "content": f"Please summarize the following news article content:\n\"{article_text}\""
    }

    # Call the Groq API to generate the summary using LLaMA-3.1-70B model
    try:
        completion = client.chat.completions.create(
            model="llama-3.1-70b-versatile",  # Use the LLaMA-3.1-70B model
            messages=[message],               # Pass the article text
            temperature=0.7,                  # Lower temperature for more focused responses
            max_tokens=1024,                  # Limit the output to 1024 tokens
            top_p=0.9                         # Control diversity of the output
        )

        # Extract the generated response
        response = completion.choices[0].message.content
        return response
    except Exception as e:
        print("Error while summarizing article:", e)
        return "Unable to summarize the article at this moment."

In [9]:
# Step 7: Convert text to speech
def text_to_speech(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save('summary_audio.mp3')
    display(Audio('summary_audio.mp3', autoplay=True))

In [10]:
# Step 8: Main function to run the notebook cell
def main():
    country, category, source, role = get_user_preferences()

    # Build a query for Google search
    query = build_query(country, category, source, role)

    # Fetch news URLs based on the search query
    news_articles = google_search_news(query)

    if news_articles:
        print("Here are the top articles based on your preferences:")
        for i, article in enumerate(news_articles, 1):
            print(f"{i}. {article['title']} - {article['url']}")

        # Ask user to select an article for summarization
        article_index = int(input("Select an article number to summarize: ")) - 1
        selected_article_url = news_articles[article_index]['url']

        # Scrape, summarize, and convert to speech
        summary = scrape_and_summarize_article(selected_article_url)
        text_to_speech(summary)
    else:
        print("No news available for the given preferences.")

# **Run this main function**

In [12]:
main()

Please select your news preferences:
Enter country code (e.g., 'us' for USA, 'in' for India): India
Enter news category (e.g., 'technology', 'business', 'entertainment', 'health'): 
Enter specific news source (e.g., 'bbc-news', 'the-hindu') or leave blank for default: 
Enter your role (e.g., 'student', 'lawyer', 'politician') for personalized news: 
Here are the top articles based on your preferences:
1. www.ndtv.com - https://www.ndtv.com/
2. indianexpress.com - https://indianexpress.com/
3. timesofindia.indiatimes.com - https://timesofindia.indiatimes.com/
4. www.hindustantimes.com - https://www.hindustantimes.com/
5. www.bbc.com - https://www.bbc.com/news/world/asia/india
Select an article number to summarize: 3
Extracted text from article:
 'Binny And Family' highlights the significance of bridging generational divides through understanding and open communication.
'Love Sitara' rests mostly on Sitara's shoulders and Sobhita Dhulipala's stunning screen presence shines through with R